In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000


In [11]:
#VAL_X = np.empty((384,TIMESTEPS,1),dtype=float)
#VAL_Y = np.empty((384,1),dtype=float)

TIME_STEPS=150
NUMBER_OF_STEPS=int(629145481/TIME_STEPS)
columns = ['acoustic_data','time_to_failure']

path = 'train.csv.zip'

def get_batch():

    while 1:
        chunks = pd.read_csv(path, float_precision='round_trip', header=0, compression='zip',
                         chunksize=TIME_STEPS, iterator=True)
        val_i = 0
        for idx, chunk in enumerate(chunks):
            batch = chunk['acoustic_data'].values.reshape(1,TIME_STEPS,1)
            target = chunk['time_to_failure'].values.reshape(-1,1)[-1]
                #print(batch.shape)
            #if idx % 10 == 0:
             #   for j,b in enumerate(batch):
              #      if val_i < 384:
               #         VAL_X[val_i] = b
                #        VAL_Y[val_i] = target[j]
                 #       val_i+=1
                #continue
            #else:
            yield (batch, target)


In [12]:
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,Conv1D,TimeDistributed,MaxPooling1D,Flatten#,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional

BATCH_SIZE=1

dropout=0.2


my_model = Sequential()


my_model.add(LSTM(use_bias = True,unit_forget_bias=True,\
                  units = 4,\
                  stateful=True,batch_input_shape=(BATCH_SIZE,TIME_STEPS,1),\
                  #dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))



my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=0),
    #Reseter()
]

#my_training_batch_generator = MY_Generator()

#####


#for i in range(0,100):
 #   print("epoch",i)
  #  big_chunks = pd.read_csv('train.csv.zip', float_precision='round_trip', header=0, compression='zip',
   #                      chunksize=BATCH_SIZE*TIMESTEPS, iterator=True)
    #epoch_losses = []
    #for j,big_chunk in enumerate(big_chunks):
     #   if j > BATCH_SIZE:
      #      continue
       # print("batch",j)
       # train, targets = get_generated_batch(big_chunk)
       # losses = my_model.train_on_batch(train, targets)
       # epoch_losses.append(losses[0])
    #print("epoch loss: ", np.mean(epoch_losses))



history = my_model.fit_generator(generator=get_batch(),
                                          steps_per_epoch=NUMBER_OF_STEPS,
                                          epochs=10,
                                          #validation_data=(VAL_X, VAL_Y),
                                          #callbacks=callbacks,
                                          #shuffle=False,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          #use_multiprocessing=False,
                                          #workers=8,
                                          #max_queue_size=32
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (1, 4)                    96        
_________________________________________________________________
dense_4 (Dense)              (1, 1)                    5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
   2326/4194303 [..............................] - ETA: 47:36:15 - loss: 0.1203 - mean_absolute_error: 0.1203

KeyboardInterrupt: 

In [ ]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [ ]:
test_splits

In [ ]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

In [ ]:
ids = []
preds = []
i=0
for r in res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

In [ ]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()